In [75]:
import wandb
import pandas as pd
import os
from tqdm import tqdm

# from table_plotter import print_result_table

In [76]:
api = wandb.Api(timeout=600)


In [77]:
# Specify cache directory
cache_dir = "./wandb_cache"
os.makedirs(cache_dir, exist_ok=True)

In [78]:
dfs = []
skipped_runs = []  # List to store IDs of skipped runs

evaluation_keys = ['Evaluation/acc_imp_perc', 'Evaluation/exist_imp_perc', 'Evaluation/reach_imp_perc', 'Evaluation/path_length',
                   'Evaluation/fn_imp_perc', 'Evaluation/fp_imp_perc', 'Evaluation/tn_imp_perc', 'Evaluation/tp_imp_perc', 
                   'Evaluation/solvability', 'Evaluation/playability']
                     

In [79]:
def get_dataframe_from_run(run):
    
    for run in tqdm(runs):
    
        # Define cache filename based on run ID
        cache_file = os.path.join(cache_dir, f"{run.id}.csv")
        
        # Check if cached file exists
        if os.path.exists(cache_file):
            # Load cached DataFrame
            df = pd.read_csv(cache_file)
        else:
            if run.state == "running":
                print(f"Skipping run ID: {run.id} (state: {run.state})")
                continue
            
            df = run.history(keys=["Evaluation/llm_iteration", *evaluation_keys[:1]])
    
            def append_key(src_df, key):
    
                tgt_df = run.history(keys=[key, "Evaluation/llm_iteration"])
                src_df = pd.merge(src_df, tgt_df, on="Evaluation/llm_iteration", how="outer")
                src_df = src_df.drop(columns=["_step_x", "_step_y"], errors="ignore")
                return src_df
    
            for key in evaluation_keys[1:]:
                try:
                    df = append_key(df, key)
                except Exception as e:
                    print(f"Error: {e} at run ID: {run.id}")
    
            
            # Add run config to DataFrame with prefix 'config.'
            for key, value in run.config.items():
                if isinstance(value, list):
                    value = ",".join(map(str, value))  # Convert list to comma-separated string
                df[key] = value
    
            # 기본값 설정
            default_values = {'n_self_alignment': 0, 'feedback_type': 'default'}
            # 열이 없을 경우 기본값으로 채워 넣기
            for col, value in default_values.items():
                if col not in df.columns:
                    df[col] = value
            
             
            # Filter columns
            key_filter = ['run_id', 'final_state', 'target_character', 'pe', 'gpt_model', 'branch_factor', 'exp_name', 'evaluator', 'total_iterations', 'n_self_alignment', 'feedback_type', 'feedback_input_type', 'total_timesteps', 
                          'reward_feature', 'fewshot', 'problem', 'seed', 
                          'Evaluation/llm_iteration'] + evaluation_keys
            auxiliary_key_filter = []
            
            df['run_id'] = run.id  # Add run ID as a column
            df['final_state'] = run.state
            
            try:
                df = df[key_filter + auxiliary_key_filter]
            except KeyError:
                df = df[key_filter]
            
            # Save DataFrame to cache as CSV
            df.to_csv(cache_file, index=False)
        
        dfs.append(df)
    
    # Concatenate all DataFrames
    df = pd.concat(dfs, ignore_index=True)
    
    return df

In [80]:
runs = api.runs("inchangbaek4907/scenario")
scenario_df = get_dataframe_from_run(runs)
scenario_df

 96%|█████████▋| 206/214 [02:21<00:39,  4.91s/it]

Skipping run ID: y21rdfaa (state: running)
Skipping run ID: aa7gds3q (state: running)
Skipping run ID: w7rl5pwx (state: running)


 98%|█████████▊| 210/214 [02:26<00:10,  2.60s/it]

Skipping run ID: p4l8wivg (state: running)


100%|██████████| 214/214 [02:30<00:00,  1.43it/s]

Skipping run ID: 576x6twm (state: running)
Skipping run ID: pw44xsft (state: running)


,run_id,final_state,target_character,pe,gpt_model,branch_factor,exp_name,evaluator,total_iterations,n_self_alignment,...,Evaluation/acc_imp_perc,Evaluation/exist_imp_perc,Evaluation/reach_imp_perc,Evaluation/path_length,Evaluation/fn_imp_perc,Evaluation/fp_imp_perc,Evaluation/tn_imp_perc,Evaluation/tp_imp_perc,Evaluation/solvability,Evaluation/playability
0,re2wd1xx,finished,1,cot,gpt-4o,2,def,hr,6,0,...,0.011111,0.533333,0.533333,26.000002,2.566667,0.400000,0.0,0.033333,0.200000,1.000000
1,re2wd1xx,finished,1,cot,gpt-4o,2,def,hr,6,0,...,0.000000,0.000000,0.000000,26.000000,2.666667,0.333333,0.0,0.000000,0.166667,0.966667
2,re2wd1xx,finished,1,cot,gpt-4o,2,def,hr,6,0,...,0.033333,0.633333,0.600000,26.214287,2.566667,0.333333,0.0,0.100000,0.166667,0.933333
3,re2wd1xx,finished,1,cot,gpt-4o,2,def,hr,6,0,...,0.000000,0.600000,0.133333,0.000000,3.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,re2wd1xx,finished,1,cot,gpt-4o,2,def,hr,6,0,...,0.000000,0.166667,0.000000,0.000000,3.000000,0.000000,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303,pprfvlvi,finished,2,got,chatgpt-4o,2,chatgpt4osa,hr,6,5,...,0.166667,0.950000,0.633333,27.500000,2.233333,0.266667,0.0,0.500000,0.233333,0.400000
1304,pprfvlvi,finished,2,got,chatgpt-4o,2,chatgpt4osa,hr,6,5,...,0.000000,0.700000,0.200000,26.000000,3.000000,0.000000,0.0,0.000000,0.000000,0.066667
1305,29dloz81,finished,2,got,chatgpt-4o,2,chatgpt4osa,hr,6,5,...,0.000000,0.566667,0.033333,0.000000,3.000000,0.000000,0.0,0.000000,0.000000,0.000000
1306,0xhn2cex,finished,2,got,chatgpt-4o,2,chatgpt4osa,hr,6,5,...,0.000000,0.616667,0.383333,26.250000,3.000000,0.000000,0.0,0.000000,0.000000,0.533333


In [81]:
# Print summary of skipped runs
print("\nSummary of Skipped Runs:")
print(f"Total skipped runs: {len(skipped_runs)}")
print("Skipped run IDs:", skipped_runs)


Summary of Skipped Runs:
Total skipped runs: 0
Skipped run IDs: []


In [82]:
df = pd.concat([scenario_df], ignore_index=True)

In [83]:
# only the target_character is '1' or '2'
df = df[df['target_character'].isin([1, 2])]

In [84]:
time_str = pd.Timestamp.now().strftime("%Y-%m-%d-%H-%M-%S")

In [85]:
df.to_csv(f"wandb_output_{time_str}.csv", index=False)